<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/text_classification/spam_vs_ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shazzadraihan","key":"852204f633bb05fc928ef0439e027938"}'}

In [2]:
# make a kaggle dir, copy the API key to it
# and make sure the file in only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# use API command to download the dataset
!kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 63.6MB/s]


In [4]:
# uncompress the dataset
!unzip -qq sms-spam-collection-dataset.zip

In [5]:
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [6]:
import torch
import torchtext
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
data = pd.read_csv("/content/spam.csv", encoding="latin-1")

data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
data = data.drop(columns = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(index = str, columns = {"v1": "labels", "v2": "text"})

data.head()

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

train.reset_index(drop=True), test.reset_index(drop=True)

(     labels                                               text
 0       ham  No I'm in the same boat. Still here at my moms...
 1      spam  (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
 2       ham     They r giving a second chance to rahul dengra.
 3       ham     O i played smash bros  &lt;#&gt;  religiously.
 4      spam  PRIVATE! Your 2003 Account Statement for 07973...
 ...     ...                                                ...
 4452    ham  I came hostel. I m going to sleep. Plz call me...
 4453    ham                             Sorry, I'll call later
 4454    ham      Prabha..i'm soryda..realy..frm heart i'm sory
 4455    ham                         Nt joking seriously i told
 4456    ham                In work now. Going have in few min.
 
 [4457 rows x 2 columns],
      labels                                               text
 0       ham  Funny fact Nobody teaches volcanoes 2 erupt, t...
 1       ham  I sent my scores to sophas and i had to do sec...
 2      spam

In [10]:
train.shape, test.shape

((4457, 2), (1115, 2))

In [11]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [12]:
!ls

kaggle.json  sms-spam-collection-dataset.zip  test.csv
sample_data  spam.csv			      train.csv


In [13]:
import nltk
nltk.download("punkt")

from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
TEXT = Field(tokenize = word_tokenize)

In [15]:
LABEL = Field(dtype = torch.float)

In [16]:
datafields = [("labels", LABEL), ("text", TEXT)]

In [17]:
trn, tst = TabularDataset.splits(path = './',
                                        train = "train.csv",
                                        test = "test.csv",
                                        format = "csv",
                                        skip_header = True,
                                        fields = datafields)

In [18]:
trn[:5]

In [19]:
print(f'Number of training examples: {len(trn)}')
print(f'Number of testing examples: {len(tst)}')

Number of training examples: 4457
Number of testing examples: 1115


In [20]:
trn[5].__dict__.keys()

dict_keys(['labels', 'text'])

In [21]:
trn[5].text

['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']

In [22]:
trn[5].labels

['ham']

In [23]:
print(vars(trn.examples[5]))

{'labels': ['ham'], 'text': ['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']}


In [24]:
TEXT.build_vocab(trn, max_size = 10500)

In [25]:
LABEL.build_vocab(trn)

In [26]:
print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary: 10213
Unique tokens in LABEL vocabulary: 4


In [27]:
print(TEXT.vocab.freqs.most_common(50))

[('.', 3847), ('to', 1750), ('I', 1574), (',', 1468), ('you', 1462), ('?', 1256), ('!', 1134), ('a', 1068), ('the', 946), ('...', 923), ('&', 772), ('i', 760), ('and', 673), ('in', 663), ('is', 647), (';', 641), ('u', 636), ('me', 600), (':', 570), ('..', 544), ('for', 527), ('my', 494), ('of', 471), ('it', 470), ('your', 461), ('have', 395), ('on', 394), (')', 393), ('2', 390), ('that', 385), ("'s", 384), ("'m", 320), ('now', 318), ('are', 316), ('do', 312), ('call', 307), ('at', 301), ('U', 300), ('or', 298), ('not', 295), ("n't", 281), ('be', 275), ('*', 270), ('lt', 267), ('gt', 267), ('with', 267), ('get', 265), ('will', 264), ('so', 257), ('#', 245)]


In [28]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '.', 'to', 'I', ',', 'you', '?', '!', 'a']


In [29]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f150a7fba10>>, {'<unk>': 0, '<pad>': 1, 'ham': 2, 'spam': 3})


In [55]:
batch_size = 64

train_iterator, test_iterator = BucketIterator.splits(
    (trn, tst),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False)

In [56]:
 import torch.nn as nn

 class RNN(nn.Module):
   def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
     super().__init__()
     self.embedding = nn.Embedding(input_dim, embedding_dim)
     self.rnn = nn.RNN(embedding_dim, hidden_dim)
     self.fc = nn.Linear(hidden_dim, output_dim)

   def forward(self, text):
     embedded = self.embedding(text)
     output, hidden = self.rnn(embedded)
     hidden_1D = hidden.squeeze(0)
     assert torch.equal(output[-1, :, :], hidden_1D)
     return self.fc(hidden_1D)

In [57]:
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [58]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 1e-6)
criterion = nn.BCEWithLogitsLoss()

In [59]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0

  model.train()

  for batch in iterator:
    optimizer.zero_grad()
    predictions = model(batch.text)
    loss = criterion(predictions, (batch.labels).T)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [63]:
num_epochs = 5

for epoch in range(num_epochs):

  train_loss = train(model, train_iterator, optimizer, criterion)

  print(f'Epoch: {epoch+1:02}  Train Loss: {train_loss:.3f}')

Epoch: 01  Train Loss: 0.367
Epoch: 02  Train Loss: 0.236
Epoch: 03  Train Loss: 0.100
Epoch: 04  Train Loss: -0.035
Epoch: 05  Train Loss: -0.170


In [64]:
epoch_loss = 0

model.eval()
with torch.no_grad():
  for batch in test_iterator:
    predictions = model(batch.text)
    loss = criterion(predictions, (batch.labels).T)
    epoch_loss += loss.item()

test_loss = epoch_loss / len(test_iterator)

print(f'Test Loss: {test_loss:.3f}')

Test Loss: 0.576
